In [2]:
import json
import pandas as pd
import numpy as np
from influence_benchmark.backend.hf_backend import HFBackend
from influence_benchmark.utils.utils import find_freest_gpus
from pathlib import Path

/nas/ucb/adhyyan/miniconda3/envs/influence/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from influence_benchmark.data_root import PROJECT_DATA
TRAJ_PATH = PROJECT_DATA / "trajectories"

In [4]:
%load_ext autoreload
%autoreload 2

**Inspect dataset**

In [6]:
dataset_filename = "/nas/ucb/adhyyan/Influence-benchmark/data/benchmarks/sycophancy/feedback.jsonl"
feedback_df = pd.read_json(dataset_filename, lines=True)

In [7]:
def split_and_truncate(feedback_df, save = False):    
# Now extract the dataset from the base column
    feedback_df['dataset'] = feedback_df['base'].apply(lambda x: x.get('dataset', None))

    # Get the unique dataset values
    unique_datasets = feedback_df['dataset'].unique()

    # Split the DataFrame based on these unique dataset values
    dfs = {}
    for dataset in unique_datasets:
        dfs[dataset] = feedback_df[feedback_df['dataset'] == dataset]
    # If you want to access each dataframe separately, for example:
    arguments_df = dfs.get('arguments')
    poems_df = dfs.get('poems')
    math_df = dfs.get('math')
    # Truncate each dataframe to its first 70 entries
    truncated_dfs = {key: df.head(70) for key, df in dfs.items()}

    # Concatenate the truncated DataFrames
    small_feedback_df = pd.concat(truncated_dfs.values(), ignore_index=True)
    if save:
        # Define the path where you want to save the JSONL file
        output_path = '/nas/ucb/adhyyan/Influence-benchmark/data/benchmarks/sycophancy/feedback_210.jsonl'
        # Save the DataFrame as a JSONL file
        small_feedback_df.to_json(output_path, orient='records', lines=True)

    return truncated_dfs, small_feedback_df